In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from preprocessing.preprocessing import DataProcessor
from utils.visualization import plot_glucose_series, plot_feature_distribution
from config.config import Config


In [ ]:
processor = DataProcessor()
demo_df = processor._load_demographics()
sample_file = list(Config.RAW_DATA.glob("Subject*.xlsx"))[0]
features, label = processor.process_file(sample_file, demo_df)
print("Extracted Features:", features)
print("Label:", label)

In [ ]:
cgm_df = pd.read_excel(sample_file, sheet_name="CGM")
fig = plot_glucose_series(cgm_df['mg/dl'], title="Sample CGM Glucose Levels")
fig.show()

In [ ]:
feature_names = ["CGM Mean", "CGM Std", "Glycemic Variability", "Age", "HbA1C"]
feature_dict = dict(zip(feature_names, features))
fig2 = plot_feature_distribution(feature_dict, title="Extracted Features")
fig2.show()

In [ ]:
import torch
from models.tabtransformer import TabTransformer

model = TabTransformer(
    num_features=Config.NUM_FEATURES,
    num_classes=Config.NUM_CLASSES,
    dim=Config.TRANSFORMER_DIM,
    depth=Config.TRANSFORMER_DEPTH,
    heads=Config.TRANSFORMER_HEADS
)
checkpoint = Config.CHECKPOINTS / "TabTransformer_best.pt"
model.load_state_dict(torch.load(checkpoint, map_location=Config.DEVICE))
model.to(Config.DEVICE)
model.eval()

tensor = torch.tensor([features], dtype=torch.float32).to(Config.DEVICE)
with torch.no_grad():
    output = model(tensor)
prediction = int(output.argmax().item())
print("Model Prediction:", prediction)